In [44]:
#Import Package
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing

#importing datasets  
df= pd.read_csv('final_grouping.csv') 


#updating all the numeric columns to float from string
df = df.astype({'LARGEST_GIFT_AMOUNT': float,'TOTAL_NUMBER_GIFTS': float,'TOTAL_GIVING_CAPACITY': float,'TOTAL_INCOME': float,'TOTAL_DONATIONS': float,'TOTAL_EAD': float,'LARGEST_REALEST_VALUE': float,'LAST_GIFT_AMOUNT': float,'age': float,'Avg_Gift_Amount': float,'gifting_Capcity': float})

df

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,age,Group
0,4,135.0,1977-06-01,2011,2011,2011,135.0,1.0,38055.0,47500.0,...,539909.0,0.0,M,135.0,135.0,False,135.000000,0.003547,45.0,0
1,21,50.0,1957-12-01,2012,2012,2012,50.0,1.0,90861.0,187500.0,...,241225.0,0.0,M,50.0,50.0,False,50.000000,0.000550,65.0,2
2,24,7200.0,1938-11-23,2000,2006,2006,18200.0,3.0,260330.0,375000.0,...,267601.0,0.0,M,6000.0,5400.0,False,6066.666667,0.069911,84.0,4
3,28,240.0,1946-10-01,2000,2004,2009,1010.0,7.0,54790.0,71000.0,...,278708.0,0.0,M,144.0,48.0,False,144.285714,0.018434,76.0,4
4,36,200.0,1980-12-01,2009,2010,2013,660.0,4.0,206862.0,125000.0,...,1046190.0,0.0,F,180.0,180.0,False,165.000000,0.003191,42.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179,26651,605.0,1982-11-01,2014,2014,2014,605.0,1.0,12505.0,70000.0,...,0.0,0.0,F,605.0,605.0,True,605.000000,0.048381,40.0,9
3180,26684,600.0,1940-08-01,2005,2012,2012,1200.0,2.0,62147.0,87500.0,...,168939.0,0.0,M,600.0,600.0,False,600.000000,0.019309,82.0,9
3181,26741,5000.0,1941-08-01,2020,2021,2021,10000.0,2.0,321380.0,187500.0,...,972820.0,0.0,F,5000.0,5000.0,False,5000.000000,0.031116,81.0,9
3182,26784,20000.0,1965-09-20,1995,2012,2020,57692.0,22.0,57692.0,0.0,...,0.0,1.0,F,40.0,250.0,False,2622.363636,1.000000,57.0,9


In [45]:
def calculate_likelihood_gaussian(df,feat_name,feat_val,y):
    mean,std=float(df[feat_name].mean()),float(df[feat_name].std())
    
    prob=(1/(np.sqrt(2.0*np.pi)*std))*np.exp(-((feat_val-mean)**2.0/(2.0*std**2.0)))
    return(prob)

In [46]:
def naive_bayes_gaussian_num(df,x,y):
    features=list(df.columns)[:-1]
    
    x_score=[]
    for i in range(len(df.iloc[:,:1])):
        result=1
        for j in range(len(features)):
            val=float(df.iloc[i,j])
            result=result*calculate_likelihood_gaussian(df,features[j],val,y)
        x_score.append(result)
        result=1
    return(x_score)

In [47]:
def naive_bayes_gaussian_cat(df,x,y):
    features=list(df.columns)[:-1]
    
    x_score=[]
    for i in range(len(df.iloc[:,:1])):
        result=1
        for j in range(len(features)):
            val=df.iloc[i,j]
            result=result*probab(df,features[j],val)
        x_score.append(result)
        result=1
    return(x_score)

In [48]:
def probab(df,feat_name,feat_val):
    prob= len(df[df[feat_name]==feat_val])/len(df)
    return(prob)

In [49]:
dff_final = pd.DataFrame()
for i in range(10):
    df1=df.loc[df['Group'] == i]
    df_num_col= df1[['LARGEST_GIFT_AMOUNT','TOTAL_NUMBER_GIFTS','TOTAL_GIVING_CAPACITY','TOTAL_INCOME','TOTAL_DONATIONS','TOTAL_EAD','LARGEST_REALEST_VALUE','LAST_GIFT_AMOUNT','Avg_Gift_Amount','gifting_Capcity','age','Group']]
    
    dff_num=df_num_col.iloc[:,:-1].values
    x_score_num=naive_bayes_gaussian_num(df_num_col,dff_num,y="Group")
    
    df_cat_col=df1[['FIRST_GIFT_DATE','LARGEST_GIFT_DATE','LAST_GIFT_DATE','TOTAL_NO_EVENTS_PARTICIPATED','GENDER','Financially_In_Dept','Group']]
    dff_cat=df_cat_col.iloc[:,:-1].values
    x_score_cat=naive_bayes_gaussian_cat(df_cat_col,dff_cat,y="Group")
    
    
    res_list = [x_score_num[i] * x_score_cat[i] for i in range(len(x_score_cat))]
    df1['Score']=res_list
   
    dff_final=dff_final.append(df1, ignore_index=True)
    res_list.clear()
    x_score_cat.clear()
    x_score_num.clear()
    
    
    

C:\Users\rathakra2201\AppData\Local\Temp\1\ipykernel_21192\3308398175.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Score']=res_list
C:\Users\rathakra2201\AppData\Local\Temp\1\ipykernel_21192\3308398175.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff_final=dff_final.append(df1, ignore_index=True)
C:\Users\rathakra2201\AppData\Local\Temp\1\ipykernel_21192\3308398175.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [50]:
dff_final.to_csv('final_Scoring.csv', header=True, index=False)